# Build an Agent - 构建代理

语言模型本身无法采取行动 - 它们只是输出文本。LangChain 的一个重要用例是创建代理。  
代理是使用 LLM 作为推理引擎来确定要采取哪些操作以及要传递给它们的输入的系统。执行操作后，结果可以反馈到 LLM 中，以确定是否需要更多操作，或者是否可以完成。

在本教程中，我们将构建一个可以与搜索引擎交互的代理。您将能够向该代理提问，观察它调用搜索工具，并与其进行对话。 

## 概念
在本教程中，您将学习如何：

 - 使用语言模型，特别是其工具调用能力
 - 使用搜索工具从互联网上查找信息
 - 编写 LangGraph 代理，使用 LLM 确定操作然后执行它们


In [9]:
# 获取你的智谱 API Key
# 在当前文件下创建一个.env文件，将api-key复制进去，如ZHIPUAI_API_KEY = "api-key"

from dotenv import load_dotenv,find_dotenv
import os 
_ = load_dotenv(find_dotenv())

## Define tools - 定义工具

我们首先需要创建我们想要使用的工具。我们选择的主要工具是 Tavily - 一个搜索引擎。我们在 LangChain 中有一个内置工具，可以轻松使用 Tavily 搜索引擎作为工具。

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2,tavily_api_key=os.environ["TAVILY_API_KEY"])
search_results = search.invoke("天津的天气怎么样?")
print(search_results)
# 如果我们愿意，我们可以创建其他工具。
# 一旦我们拥有了所有想要的工具，我们就可以将它们放入一个列表中。
tools = [search]

[{'url': 'http://www.weather.com.cn/weather/101030100.shtml', 'content': '涂擦SPF大于15、PA+防晒护肤品。\n天气资讯\n周边地区\n|\n周边景点\n2024-02-25 18:00更新\n/\n/\n/\n/\n/\n/\n周边地区\n|\n周边景点\n2024-02-25 18:00更新\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n/\n高清图集\n重大天气事件\n2月25日\n湖南湖北等地冻雨今天减弱 南方多地冷如三九\n今天（2月25日），湖南、湖北等地的冻雨天气有所减弱，范围缩小，但长江沿线及其以南地区多地仍有降水，且相态复杂，公众需关注预警预报信息。\n2月24日\n南方阴雨天气延续至3月初 周末江南等地气温明显偏低\n本周末，北方降水明显减少，南方仍有较大范围阴雨雪天气，湖南、贵州等地仍需防范冻雨。预计，南方大部阴雨频繁的天气将延续至3月初。\n2月23日\n北方降雪明日基本停歇 南方多地月底前气温持续偏低\n今天（2月23日），我国中东部等地大范围雨雪持续，降水强度继续减弱。明后天，北方大部降水停歇，南方仍有雨雪天气。\n2月22日\n中东部降雪区域向南收缩 南方大部气温持续低迷\n今明天（2月22日至23日），我国雨雪有所减弱，雨雪分界线继续南压。后天，北方大部降雪基本停歇，而南方阴雨雪天气还将持续到本月底。\n2月21日\n中东部雨雪冰冻天气持续 南方多地湿冷感十足\n今天（2月21日），中东部雨雪冰冻天气持续，同时气温还将继续下降，江淮、江汉、江南等地在雨雪的“打压”下，体感尤为湿冷。\n2月20日\n中东部雨雪进入最强时段 南方多地气温陆续跳水\n受寒潮影响，今明两天（2月20日至21日），中东部雨雪进入最强时段。同时，未来四天，中东部气温继续下降，多地将上演冷暖大逆转。\n2月19日\n中东部将遭遇剧烈降温 明起雨雪天气进入鼎盛阶段\n今天（2月19日），寒潮主要影响我国北方地区，明天起，中东部雨雪天气将进入鼎盛阶段。除了雨雪，今起四天，\u200b我国大部地区将迎来剧烈降温。\n2月18日\n寒潮发威北方多地开启猛烈降温模式 大范围雨雪天气逐渐铺展\n今起（2月18日）至22日，龙年首场寒潮继续东移南下，北方多地开启降温模式，新疆、青

# Using Language Models 使用语言模型

接下来，让我们学习如何使用语言模型来调用工具。LangChain 支持许多不同的语言模型,我们用的清华的智谱模型。

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key=os.environ["ZHIPUAI_API_KEY"],
    model="glm-4",
)

您可以通过传入消息列表来调用语言模型。默认情况下，响应是内容字符串。

In [16]:
from langchain_core.messages import HumanMessage

resp = model.invoke([HumanMessage(content="你好。")])
resp.content

'你好！有什么我可以帮助你的吗？'

现在，我们可以看看如何让这个模型进行工具调用。为了实现这一点，我们使用 .bind_tools 为语言模型提供这些工具的知识

In [17]:
model_with_tools = model.bind_tools(tools)

现在我们可以调用该模型了。我们先用普通消息调用它，看看它如何响应。我们可以查看 content 字段以及 tool_calls 字段。

In [18]:
resp = model_with_tools.invoke([HumanMessage(content="你好。")])

print(f"ContentString: {resp.content}")
print(f"ToolCalls: {resp.tool_calls}")

ContentString: 你好，请问有什么可以帮助你的吗？
ToolCalls: []


现在，让我们尝试使用一些需要调用工具的输入来调用它。

In [30]:
resp = model_with_tools.invoke([HumanMessage(content="天津明天的限号是多少?")])

print(f"ContentString: {resp.content}")
print(f"ToolCalls: {resp.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '天津明天限号是多少'}, 'id': 'call_8741766118450732492'}]


我们可以看到现在没有文本内容，但有一个工具调用！它希望我们调用 Tavily Search 工具。

这还没有调用该工具 - 它只是告诉我们这样做。为了真正调用它，我们需要创建我们的代理。

## Create the agent - 创建代理

现在我们已经定义了工具和 LLM，我们可以创建代理了。我们将使用 LangGraph 来构建代理。  
目前，我们使用高级接口来构建代理，但 LangGraph 的优点在于，如果您想要修改代理逻辑，这个高级接口由低级、高度可控的 API 支持。

现在，我们可以使用 LLM 和工具初始化代理。

请注意，我们传入的是模型，而不是 model_with_tools。这是因为 create_react_agent 会在后台为我们调用 .bind_tools。

### 安装 LangGraph¶

In [32]:
# pip install -U langgraph

In [33]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model,tools)

## Run the agent - 运行代理

我们现在可以在几个查询上运行代理！请注意，目前这些都是无状态查询（它不会记住以前的交互）。  
请注意，代理将在交互结束时返回最终状态（其中包括任何输入，我们稍后将了解如何仅获取输出）。

首先，让我们看看当不需要调用工具时它如何响应：

In [34]:
response = agent_executor.invoke({"messages": [HumanMessage(content="你好。")]})

response["messages"]

[HumanMessage(content='你好。', id='ea11dcb7-bbd9-49b5-91c6-e911c9b3f867'),
 AIMessage(content='你好，请问有什么可以帮助您的吗？', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 139, 'total_tokens': 149}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bd7e5736-411c-44ad-8a41-4f4017f59291-0')]

现在让我们在一个应该调用该工具的示例上尝试一下

In [36]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="天津明天6月17日的限号是多少?")]}
)
response["messages"]

[HumanMessage(content='天津明天6月17日的限号是多少?', id='74a1d228-f7a8-4da4-b9a8-3a99bf09751f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8741766255889691704', 'function': {'arguments': '{"query":"天津明天6月17日的限号是多少"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 147, 'total_tokens': 173}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87044df6-f191-482d-bf2e-0b5c31e69ae4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '天津明天6月17日的限号是多少'}, 'id': 'call_8741766255889691704'}]),
 ToolMessage(content='[{"url": "http://tj.bendibao.com/traffic/2024325/144128.shtm", "content": "2024\\u5929\\u6d25\\u6700\\u65b0\\u9650\\u884c \\u9650\\u53f7\\u5b89\\u6392\\u5df2\\u516c\\u5e03\\uff082024.4.1-2025.3.30\\uff09 \\u81ea2024\\u5e744\\u67081\\u65e5\\u81f32025\\u5e743\\u670830\\u65e5 \\uff0c\\u5de5\

## Streaming Messages - 流式消息
我们已经了解了如何使用 .invoke 调用代理来获取最终响应。如果代理正在执行多个步骤，则可能需要一段时间。  
为了显示中间进度，我们可以在消息发生时流式返回消息。

In [39]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="2024年6月17日，天津的限号是多少?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8741760723971565035', 'function': {'arguments': '{"query":"2024年6月17日天津限号是多少"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 151, 'total_tokens': 179}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9c342ec8-ddf2-4014-9557-bbcd084967e1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024年6月17日天津限号是多少'}, 'id': 'call_8741760723971565035'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "http://tj.bendibao.com/traffic/2024325/144129.shtm", "content": "\\u5de5\\u4f5c\\u65e5\\u6bcf\\u65e57\\u65f6\\u81f319\\u65f6\\uff0c\\u5929\\u6d25\\u5b9e\\u65bd\\u6309\\u8f66\\u724c\\u5c3e\\u53f7\\u533a\\u57df\\u9650\\u884c\\u4ea4\\u901a\\u7ba1\\u7406\\u63aa\\u65bd\\u3002\\u81ea2024\\u5e744\\u67081\\u65e5\\u

## Streaming tokens - 流式传输令牌
除了流式传输消息之外，流式传输令牌也很有用。我们可以使用 .astream_events 方法来实现这一点。

In [44]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="2024年6月17日，天津的限号是多少?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (event["name"] == "Agent"):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(f"Starting agent: {event['name']} with input: {event['data'].get('input')}")
    elif kind == "on_chain_end":
        if ( event["name"] == "Agent"):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}" )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': '2024年6月17日天津限号是多少'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'http://m.tj.bendibao.com/news/xianxingchaxun/', 'content': '不限号. 限行时间. 尾号限行：. ☞工作日每日7时至19时（因法定节假日放假调休而调整为上班的星期六、星期日除外）. 早晚高峰限行：. ☞工作日每日7时至9时、16时至19时（因法定节假日放假调休而调整为上班的星期六、星期日除外）. 限行区域. 天津外环线 ...'}, {'url': 'http://tj.bendibao.com/traffic/2024325/144128.shtm', 'content': '2024天津最新限行 限号安排已公布（2024.4.1-2025.3.30） 自2024年4月1日至2025年3月30日 ，工作日（因法定节假日放假调休而调整为上班的星期六、星期日除外）每日7时至19时，本市及外埠号牌机动车在外环线（不含）以内道路，继续实施按车牌尾号区域限行交通管理措施。'}]
--
根据|202|4|年|6|月|17|日的|日期|，|我们可以|通过|查看|限|号|安排|来确定|天津|的|限|号|。|但是|，|由于|限|号|安排|可能会|根据|法定|节假日|和|调|休|进行调整|，|因此|我们|还需要|考虑|这些|因素|。|从|搜索|结果|中|可以看出|，|限|号|安排|已经|调整|，|但是|没有|找到|202|4|年|6|月|17|日的|具体|限|号|信息|。|因此|，|我|建议|您|在|接近|那个|日期|时|再|查询|限|号|安排|，|以|获取|最|准确|的信息|。|

## Adding in memory - 添加内存
如前所述，此代理是无状态的。这意味着它不记得以前的交互。  
为了给它内存，我们需要传入一个检查点。传入检查点时，我们还必须在调用代理时传入一个thread_id（这样它就知道从哪个线程/对话恢复）。

In [45]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [46]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [47]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你好 我是张三!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，张三！有什么可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 142, 'total_tokens': 154}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b31f3e74-fd39-41aa-abf2-160facbdb41d-0')]}}
----


In [49]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你知道我叫什么吗?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='是的，你之前说过你叫张三。如果你有其他问题或需要帮助，请随时告诉我！', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 193, 'total_tokens': 216}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-29a80792-c452-4fdd-a707-080eb7555678-0')]}}
----


如果我想开始新的对话，我所要做的就是更改使用的thread_id

In [50]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你知道我叫什么吗?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='作为人工智能助手，我没有访问您个人信息的能力，所以我不知道您的名字。如果您需要个性化的帮助，请告诉我您希望如何称呼您。', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 142, 'total_tokens': 172}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9834b61c-68cd-4e3e-8885-063b52dbae48-0')]}}
----
